# BERT를 이용한 IMDB Dataset 감정분석 및 RNN과의 비교

필요한 모듈을 설치하고 불러온다.

In [ ]:
!pip install tokenizers
!pip install transformers

import io
import os
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel
from google.colab import files

max_len = 150 

사전 학습된 BERT 모델의 WordPiece를 사용한 vocab후보 목록을 다운고 Tokenizing을 위한 함수를 정의한다..

In [ ]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

save_path = "bert_base_uncased/"

if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

Fine-tuning 할 BERT 모델을 정의한다.

In [ ]:
encoder = TFBertModel.from_pretrained("bert-base-uncased")

input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
embedding = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]

x = layers.Dense(1,activation = 'sigmoid')(embedding)

bert_model = keras.Model([input_ids, token_type_ids, attention_mask],outputs=x)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


옵티마이저와 손실함수를 정의한다.


In [ ]:
optimizer = keras.optimizers.Adam(lr=5e-5)
bert_model.compile(optimizer=optimizer,loss = 'binary_crossentropy',metrics=['accuracy'])

IMDB Dataset.csv파일을 불러온다.

In [ ]:
data = pd.read_csv("/IMDB Dataset.csv")

문장과 그에 따른 감정을 나눈다.

In [ ]:
x = data['review']
y = data['sentiment']

In [ ]:
y = y.replace('positive', 1)
y = y.replace('negative', 0)

In [ ]:
y = np.array(y)
x = np.array(x)

문장의 Tokenizing을 실시한다.

In [ ]:
tokenized_X_train = []
for j in range(len(x)):
    tokenized_X_train.append(tokenizer.encode(x[j]))

In [ ]:
attention_mask = []
input_ids = []
token_type_ids = []
for i in range(len(tokenized_X_train)):
    input_ids.append(tokenized_X_train[i].ids)
    attention_mask.append([1] * len(input_ids[i]))
    token_type_ids.append([0] * len(input_ids[i]))

Padding을 해준다.

In [ ]:
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids,150,value=0)
attention_mask = tf.keras.preprocessing.sequence.pad_sequences(attention_mask,150,value=0)
token_type_ids = tf.keras.preprocessing.sequence.pad_sequences(token_type_ids,150,value=0)

epochs 수만큼 BERT 모델 훈련을 진행한다.

In [ ]:
bert_model.fit([input_ids[:49000],token_type_ids[:49000],attention_mask[:49000]],y[:49000],16,epochs=3)

Epoch 1/3
3063/3063 [==============================] - 1020s 333ms/step - loss: 0.2552 - accuracy: 0.8940
Epoch 2/3
3063/3063 [==============================] - 1017s 332ms/step - loss: 0.1403 - accuracy: 0.9475
Epoch 3/3
3063/3063 [==============================] - 1016s 332ms/step - loss: 0.0776 - accuracy: 0.9728


BERT 모델 테스트를 진행한다.

In [ ]:
bert_model.evaluate([input_ids[49000:],token_type_ids[49000:],attention_mask[49000:]],y[49000:])

32/32 [==============================] - 6s 196ms/step - loss: 0.2592 - accuracy: 0.9156


[0.2591608762741089, 0.9156267046928406]

비교를 위한 RNN기반 LSTM모델을 정의한다.

In [ ]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=100000,
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

LSTM 모델의 옵티마이저와 손실함수를 정의한다.

In [ ]:
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),loss = 'binary_crossentropy',metrics=['accuracy'])

epochs 수만큼 LSTM 모델 훈련을 진행한다.

In [ ]:
lstm_model.fit(input_ids[:49000],y[:49000],epochs=3)

Epoch 1/3
1532/1532 [==============================] - 967s 631ms/step - loss: 0.9488 - accuracy: 0.5771
Epoch 2/3
1532/1532 [==============================] - 984s 643ms/step - loss: 0.5479 - accuracy: 0.7431
Epoch 3/3
1532/1532 [==============================] - 986s 644ms/step - loss: 0.4367 - accuracy: 0.8439


LSTM 모델 테스트를 진행한다.

In [ ]:
lstm_model.evaluate(input_ids[49000:],y[49000:])

32/32 [==============================] - 2s 75ms/step - loss: 0.5271 - accuracy: 0.8520


[0.5271170139312744, 0.8519999980926514]

참고 문헌

BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding
Jacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova
Google AI Language : https://arxiv.org/abs/1810.04805

RNN을 사용한 텍스트 분류: https://www.tensorflow.org/tutorials/text/text_classification_rnn

bert 를 사용한 imdb데이터셋 학습 / 작성자 qwopqwop200
: https://blog.naver.com/qwopqwop200/222046380087

